In [ ]:
import ee
import geemap

m = geemap.Map()

CashewC_AOI = ee.FeatureCollection("projects/ee-frednumbisi/assets/CDI_CashewAOI")
CIV = ee.FeatureCollection("projects/ee-frednumbisi/assets/CIV_Boundary0")
m

In [ ]:
import ee
import geemap

m = geemap.Map()

# Import a protected areas point feature collection.
#wdpa = ee.FeatureCollection("WCMC/WDPA/current/points")

# Import a country boundaries.
countriesUSDOS = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
countriesFAO = ee.FeatureCollection("FAO/GAUL/2015/level0")
# Define a function to print metadata column names and datatypes. This function
# is intended to be applied by the `evaluate` method which provides the
# function a client-side dictionary allowing the 'columns' object of the
# feature collection metadata to be subset by dot notation or bracket notation
# (`tableMetadata['columns']`).
def getCols(tableMetadata):
    print(tableMetadata.columns.getInfo())

# Fetch collection metadata (`.limit(0)`) and apply the
# previously defined function using `evaluate()`. The printed object is a
# dictionary where keys are column names and values are datatypes.
#wdpa.limit(0).evaluate(getCols)

countriesUSDOS.limit(0).evaluate(getCols)
countriesFAO.limit(0).evaluate(getCols)

Ccodes = countriesUSDOS.aggregate_array('country_co')
Cnames = countriesUSDOS.aggregate_array('country_na')

keys = ee.Dictionary(Ccodes).keys()

print(keys.getInfo())

values = ee.Dictionary(Cnames).values()

print(values.sort().getInfo())

m

In [ ]:

# Import a protected areas point feature collection.
#wdpa = ee.FeatureCollection("WCMC/WDPA/current/points")

# Import a country boundaries.
countriesUSDOS = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
countriesFAO = ee.FeatureCollection("FAO/GAUL/2015/level0")
# Define a function to print metadata column names and datatypes. This function
# is intended to be applied by the `evaluate` method which provides the
# function a client-side dictionary allowing the 'columns' object of the
# feature collection metadata to be subset by dot notation or bracket notation
# (`tableMetadata['columns']`).
def getCols(tableMetadata):
    print(tableMetadata.columns.getInfo())

# Fetch collection metadata (`.limit(0)`) and apply the
# previously defined function using `evaluate()`. The printed object is a
# dictionary where keys are column names and values are datatypes.
#wdpa.limit(0).evaluate(getCols)

countriesUSDOS.limit(0).evaluate(getCols)
countriesFAO.limit(0).evaluate(getCols)

Ccodes = countriesUSDOS.aggregate_array('country_co')
Cnames = countriesUSDOS.aggregate_array('country_na')

keys = ee.Dictionary(Ccodes).keys()

print(keys.getInfo())

values = ee.Dictionary(Cnames).values()

print(values.sort().getInfo())


In [ ]:

#CALCULATE FOREST LOSS FOR THE WHOLE OF CDI
#

# Load the data and select the bands of interest.
# Global Forest Cover Data from 2000 to 2021 based on Landsat Imagery

gfc2021 = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

# Displaying forest, loss, gain, and pixels where both loss and gain occur.
lossImage = gfc2021.select(['loss'])
gainImage = gfc2021.select(['gain'])
treeCover = gfc2021.select(['treecover2000'])

# Use the and() method to create the lossAndGain image.
gainAndLoss = gainImage.And(lossImage)

# Set threshold of Canopy cover percentage (e.g. 10%).
coverthreshold = ee.Number(10)

# Minimum forest area in pixels (e.g. 6 pixels for Landsat 30m resolution, ~ 0.5 ha in this example).
# i.e. the Minimum Mapping Unit (MMU) of 0.5ha is represented by
pixels = ee.Number(6)

# Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = ee.Number(6)

#Apply the minimum canopy cover percentage (e.g. greater than or equal to 10%).
# Use selfMask() to set other other areas transparent by assigning value zero.

treeCover10 = treeCover.gte(coverthreshold).selfMask()

#3)  Apply the minimum area requirement using connectedPixelCount
# (e.g. greater than or equal to 6 pixels). Note that if no kernel is passed
# to connectedPixelCount, 8 neighbor adjacency is used to determine connectivity.

# Use connectedPixelCount() to get contiguous area.
contArea = treeCover10.connectedPixelCount()

# Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

# Add the threholded tree cover (treeCover10) layer in green.
m.addLayer(treeCover10.updateMask(treeCover10),
{"palette": ['000000', '00FF00'], "max": 100}, 'Forest Cover')

# Add the loss layer in red.
m.addLayer(lossImage.updateMask(lossImage),
{"palette": ['FF0000']}, 'Loss')

m

In [ ]:

#COMPUTE FOREST LOSS FOR A REGION OF INTEREST

# Load country features from Large Scale International Boundary (LSIB) dataset.
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

# Subset the Congo Republic feature from countries.
#CIV = countries.filter(ee.Filter.eq('country_na', 'Rep of the Congo'))
CIV = countries.filter(ee.Filter.eq('country_na', 'Cote d'Ivoire'))

# Sum the values of forest loss pixels in the region of interest.
stats = lossImage.reduceRegion(
"reducer"=ee.Reducer.sum(),
"geometry"=CIV,
"scale"=30,
"maxPixels"=1e9
)

print('pixels representing "loss": ', stats.get('loss').getInfo())


In [ ]:

#COMPUTE TREE COVER LOSS FOR A REGION OF INTEREST

m.addLayer(gfc2021.clip(CIV), {"bands": ['treecover2000']}, 'treecover2000')
m.centerObject(CIV, 7)

#m.addLayer(lossImage.clip(CIV))
#m.centerObject(CIV, 7)

# Select 'treecover2000' in the Global Forest Change dataset.

TreeCover2000 = gfc2021.select(['treecover2000']).clip(CashewC_AOI);

TreeCoverArea = TreeCover2000.multiply(ee.Image.pixelArea()).divide(10000)
TreeCoverSize = TreeCoverArea.reduceRegion(
"reducer"=ee.Reducer.sum(),
#"geometry"=selected.geometry(),
"geometry"=CashewC_AOI.geometry(),
"scale"=30,
"maxPixels"=1e13
)
print(
'Year 2000 Tree cover (ha.getInfo())
TreeCoverSize.get('treecover2000'))

Export.image.toDrive(
"image"=treeCover10.clip(CashewC_AOI),
"description"='CashewAOI_ForestCover10In2000',
"scale"=30,
"region"=CashewC_AOI,
"folder"='CDI_NDVI',
"maxPixels"=1e13,
"crs"='"EPSG" =3857',
"fileFormat"='GeoTIFF',
"formatOptions"={
    "cloudOptimized"=True
}

)

# Get Tree Loss in the Global Forest Change dataset

treeLoss = gfc2021.select(['lossyear'])
print(treeLoss.getInfo())

treeLoss01 = treeLoss.eq(1).selfMask();

treeLoss11 = treeLoss.lt(12).selfMask();
treeLoss15 = treeLoss.lt(16).selfMask();
treeLoss21 = treeLoss.lte(21).selfMask();

# Select the tree loss within the derived tree cover
# (>= canopy cover and area requirements).
treecoverLoss01 = TreeCover2000.And(treeLoss01).rename('loss2001').selfMask()
treecoverLoss11 = TreeCover2000.And(treeLoss11).rename('loss2011').selfMask()
treecoverLoss15 = TreeCover2000.And(treeLoss15).rename('loss2015').selfMask()
treecoverLoss21 = TreeCover2000.And(treeLoss21).rename('loss2021').selfMask()

# Add the threholded tree cover (for AOI) layer in green.
m.addLayer(TreeCover2000.updateMask(TreeCover2000).selfMask().clip(CashewC_AOI),
{"palette": ['000000', '00FF00'], "max": 100}, 'Project Area 2000 Tree Cover')
m.centerObject(CashewC_AOI, 7)

#2. Apply the minimum mapping unit using connectedPixelCount().

# Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss01.connectedPixelCount()
contLoss11 = treecoverLoss11.connectedPixelCount()
contLoss15 = treecoverLoss15.connectedPixelCount()
contLoss21 = treecoverLoss21.connectedPixelCount()

#3. Calculate the tree loss area (ha).

lossArea = treecoverLoss01.multiply(ee.Image.pixelArea()).divide(10000)
lossSize = lossArea.reduceRegion(
"reducer"=ee.Reducer.sum(),
#"geometry"=selected.geometry(),
"geometry"=CashewC_AOI.geometry(),
"scale"=30,
"maxPixels"=1e13
)
print(
'Year 2001 Tree Cover loss (ha.getInfo())
lossSize.get('loss2001'))

# Cummulative Forest Cover Loss (Area) in 2011

lossArea11 = treecoverLoss11.multiply(ee.Image.pixelArea()).divide(10000)
lossSize11 = lossArea11.reduceRegion(
"reducer"=ee.Reducer.sum(),
#"geometry"=selected.geometry(),
"geometry"=CashewC_AOI.geometry(),
"scale"=30,
"maxPixels"=1e13
)
print(
'Year 2011 Tree Cover loss (ha.getInfo())
lossSize11.get('loss2011'))

# Cummulative Forest Cover Loss (Area) in 2011

lossArea15 = treecoverLoss15.multiply(ee.Image.pixelArea()).divide(10000)
lossSize15 = lossArea15.reduceRegion(
"reducer"=ee.Reducer.sum(),
#"geometry"=selected.geometry(),
"geometry"=CashewC_AOI.geometry(),
"scale"=30,
"maxPixels"=1e13
)
print(
'Year 2015 Tree Cover loss (ha.getInfo())
lossSize15.get('loss2015'))

# Cummulative Forest Cover Loss (Area) in 2021

lossArea21 = treecoverLoss21.multiply(ee.Image.pixelArea()).divide(10000)
lossSize21 = lossArea21.reduceRegion(
"reducer"=ee.Reducer.sum(),
#"geometry"=selected.geometry(),
"geometry"=CashewC_AOI.geometry(),
"scale"=30,
"maxPixels"=1e13
)
print(
'Year 2021 Tree Cover loss (ha.getInfo())
lossSize21.get('loss2021'))

# Add the Tree cover Loss (for AOI) layer in red.
m.addLayer(treecoverLoss01.updateMask(treecoverLoss01).clip(CashewC_AOI),
{"palette": ['FF0000']}, 'ProjectArea_TCLoss2001')
m.centerObject(CashewC_AOI, 7)

m.addLayer(treecoverLoss11.updateMask(treecoverLoss11).clip(CashewC_AOI),
{"palette": ['FF0000']}, 'ProjectArea_TCLoss2011')
m.centerObject(CashewC_AOI, 7)

# applying selfMask
m.addLayer(treecoverLoss21.selfMask().clip(CashewC_AOI),
{"palette": ['FF0000']}, 'ProjectArea_TCLoss2021')
m.centerObject(CashewC_AOI, 7)

# Export Tree Cover Loss Images

Export.image.toDrive(
"image"=treecoverLoss01.clip(CashewC_AOI),
"description"='CashewAOI_TreeCoverLossIn2001b',
"scale"=30,
"region"=CashewC_AOI,
"folder"='CDI_NDVI',
"maxPixels"=1e13,
"crs"='"EPSG" =3857',
"fileFormat"='GeoTIFF',
"formatOptions"={
    "cloudOptimized"=True
}

)

Export.image.toDrive(
"image"=treecoverLoss11.clip(CashewC_AOI),
"description"='CashewAOI_TreeCoverLossIn2011b',
"scale"=30,
"region"=CashewC_AOI,
"folder"='CDI_NDVI',
"maxPixels"=1e13,
"crs"='"EPSG" =3857',
"fileFormat"='GeoTIFF',
"formatOptions"={
    "cloudOptimized"=True
}

)

Export.image.toDrive(
"image"=treecoverLoss15.clip(CashewC_AOI),
"description"='CashewAOI_TreeCoverLossIn2015b',
"scale"=30,
"region"=CashewC_AOI,
"folder"='CDI_NDVI',
"maxPixels"=1e13,
"crs"='"EPSG" =3857',
"fileFormat"='GeoTIFF',
"formatOptions"={
    "cloudOptimized"=True
}

)

Export.image.toDrive(
"image"=treecoverLoss21.clip(CashewC_AOI),
"description"='CashewAOI_TreeCoverLossIn2021b',
"scale"=30,
"region"=CashewC_AOI,
"folder"='CDI_NDVI',
"maxPixels"=1e13,
"crs"='"EPSG" =3857',
"fileFormat"='GeoTIFF',
"formatOptions"={
    "cloudOptimized"=True
}

)

m